# <center> VIPER PTA Summer School<br><br>Overlap Reduction Function Tutorial

# Packages to Install...

In [ ]:
'''
!pip install corner
!pip install nestle
!pip install scipy
!pip install astropy
!pip install sympy
'''
'''
#Optional:
!pip install jupyterthemes
!jt -t grade3 -cellw 95% -T
''';

# Loading Packages

In [ ]:
import numpy as np
from NimeDemoUtils import *
import corner
from itertools import combinations
import matplotlib.pyplot as plt
import nestle
import warnings
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format='retina'
plt.style.use('dark_background')
warnings.filterwarnings('ignore')
colors = ["aqua", "tomato", "gold", "lawngreen", "white", "magenta", "chocolate", "orange", 'green', 'gray']

# <center> Section 1: The Simplest Case of a PTA!

## Imagining a PTA

In [ ]:
Npulsars = 67
n_realization = 10
seeds = np.linspace(10000, 10062300, n_realization, dtype = int)
disttype = 'uniform'
start_obs = 53000
end_obs = 57748.25
toas = np.array([np.arange(start_obs,end_obs, 30)] * Npulsars)

In [ ]:
lam, beta, pname, xiMat = PulsarDistMaker(Npulsars = Npulsars, 
                                          seed = seeds[0], 
                                          skyplot = True, 
                                          disttype = disttype)

## Injecting a GWB

In [ ]:
res_gw = np.zeros((n_realization, Npulsars, len(toas[0])))
for ii, seed in enumerate(seeds):
    print('Making Realization {0}/{1}...'.format(ii + 1, n_realization), end = '\r')
    res_gw[ii,:,:] = GWBInj(Amp = 2e-15,
           start_obs = start_obs, 
           end_obs = end_obs, 
           Npulsars = Npulsars, 
           ang = xiMat.flatten(), 
           seed = seed, 
           toas = toas)

## Extracting the Cross Correlations

## $$\large \text{ORF} \propto \left\langle {R_a R_b} \right\rangle$$

### Finding the Cross Indices

In [ ]:
Npairs = int(Npulsars * (Npulsars - 1)/2)
ab = list(combinations(np.arange(0,Npulsars),2))
ab

### Calculating $\left\langle {r_i r_j} \right\rangle$ for Each Realization

In [ ]:
sum_res = np.zeros((n_realization, Npairs))
xiab = np.zeros(Npairs)
for rr in range(n_realization):
    for kk in range(Npairs):  
        a,b = ab[kk]
        if rr == 0: xiab[kk] = xiMat[a,b]
        sum_res[rr,kk] = np.sum(res_gw[rr, a,:]*res_gw[rr, b,:])
rho = np.mean(sum_res,axis = 0)
sig = np.std(sum_res, axis = 0)
xi = xiab.T

## Analyzing the Obtained Cross Correlation Values

### Finding the Normalization Constant For the Best HD Fit

In [ ]:
ii = 0; flag = True
while flag:
    try:
        print('{} Time Trying...'.format(ii+1))
        norm = 1e-11 #Making the Sampler not to deal with very small numbers!
        fullmodel = CrossModel(rho/norm, sig/norm, xi, pmin=[-10], model='findhdnorm',
                         pmax=[10])
        fullresult = nestle.sample(fullmodel.get_loglike,
                                   fullmodel.get_prior_transform,
                                   ndim = 1,
                                   npoints=300, dlogz=0.1)
        flag = False
    except:
        ii += 1
        continue

In [ ]:
plt.figure(dpi = 140)
plt.hist(fullresult.samples, bins=40, weights = fullresult.weights, histtype = 'step', color = 'aqua');
mean = np.average(fullresult.samples.flatten(), weights = fullresult.weights)
plt.axvline(mean, label = 'mean = {}'.format(round(mean, 2)), ls = '--', color = 'tomato')
plt.legend()
plt.xlabel('Best Fit Normalization Constant (*{})'.format(norm))
plt.ylabel('PDF');

### Binning the Calculated Cross Correlation Values

In [ ]:
xi_mean, xi_err, rho_avg, sig_avg = binned_corr_Maker(xi, rho/norm, sig/norm, nbins = 13)

### Finally, Plotting the Cross Correlation Values

In [ ]:
plt.figure(dpi = 170)
plt.errorbar(xi_mean*180/np.pi, rho_avg, xerr=xi_err*180/np.pi, yerr=sig_avg, marker='o', ls='', label = 'Estimated Correlations',alpha = .6,
                            color='aqua', capsize=4, elinewidth=1.2)
ang = np.linspace(0.001,np.pi,100)
plt.plot(ang * 180/np.pi, HD(ang) * mean, ls = '--', lw = 3, color = 'tomato', label = 'Best HD-like Fit')
plt.legend()
plt.ylabel('Normalized ORF')
plt.xlabel('Angular Separation');
plt.title('Number of Pulsars : {0}\n Distribution Type : {1}\n Number of Realizations : {2}'.format(Npulsars, disttype, n_realization))
plt.tight_layout()
plt.savefig('../Image/Corr_{0}_{1}_{2}.png'.format(Npulsars, disttype, n_realization), dpi = 300)

## Does the above plot look good to you? 

### Parameterizing HD

In [ ]:
def gt( x, tau, mono ) :
    cos_ang = np.cos(x)
    k = 1/2*(1-cos_ang)
    return 1/8 * (3+cos_ang) + (1-tau)*3/4*k*np.log(k) + mono

In [ ]:
taus = [-1.5, -1, 0, .5, 1.5]
ang = np.linspace(0.001,np.pi,100)
plt.figure(dpi = 140)
for ii, tau in enumerate(taus):
    if tau == -1: label = 'tau = -1 (HD)'
    else: label = 'tau = {}'.format(tau)
    plt.plot(ang, gt(ang, tau, 0),lw = 3, label = label, color = colors[ii])
plt.ylabel('Parameterized ORF')
plt.xlabel('Angular Separation');
plt.legend()
plt.tight_layout()

In [ ]:
fullmodel = CrossModel(rho/norm, sig/norm, xi, pmin=[-10, -2], model='gt',
                 pmax=[10, 2])
fullresult = nestle.sample(fullmodel.get_loglike,
                           fullmodel.get_prior_transform,
                           2,
                           npoints=300, dlogz=0.1)

In [ ]:
fig = corner.corner(fullresult.samples, bins=40,
                 labels = ['norm', 'tau'], color = 'aqua', truths = ['', -1], truth_color = 'tomato',
                 show_titles = True, weights=fullresult.weights)
plt.savefig('../Image/CornerCorr_{0}_{1}_{2}.png'.format(Npulsars, disttype, n_realization), dpi = 300)

# ***Question 1***:  Do we have more than 1 realization in reality? If not, how is it possible to detect the HD curve?

# ***Question 2***: keeping the number of realizations the same, what affects the detectability of correlations the most?

# <center> Section 2: Do You Know Your Integrals?

In [ ]:
from sympy import *
import sympy

## The Coordinate System

![alt text](../Image/coordinate.png "The Coordinate System")

## Setting up the Problem Using Sympy

In [ ]:
## t is theta, p is phi, and d is delta (angular separation between two pulsars)
t, p, gamma = sympy.symbols('theta phi gamma')
##The Most General Z-axis (***it is the same as the negative of k_hat from the lecture notes***)
omega = [sin(t) * cos(p), sin(t) * sin(p), cos(t)]
##The Most General X-axis
m = [cos(t)*cos(p),cos(t)*sin(p),-sin(t)] 
##The Most General Y-axis
n = [-sin(p),cos(p),0]
##Location of Pulsar a
u_a = [0,0,1]
##Location of Pulsar b
u_b = [sin(gamma),0,cos(gamma)]

## Do you remember this?

$$\large \begin{align}
  & R_{a}^{A}\left( f,\hat{k} \right)=\frac{1}{2}\frac{u_{a}^{i}u_{a}^{j}}{1-\hat{k}\cdot {{{\hat{u}}}_{a}}}e_{ij}^{A}\left( {\hat{k}} \right)\times \left[ ... \right] \\ 
 & R_{b}^{A}\left( f,\hat{k} \right)=\frac{1}{2}\frac{u_{b}^{i}u_{b}^{j}}{1-\hat{k}\cdot {{{\hat{u}}}_{b}}}e_{ij}^{A}\left( {\hat{k}} \right)\times \left[ ... \right] \\ 
\end{align}
$$

## Make Some Substitution

$$\large \begin{align}
  & B=\frac{1}{2}\frac{u_{a}^{i}u_{a}^{j}}{1-\hat{k}\cdot {{{\hat{u}}}_{a}}}e_{ij}^{A}\left( {\hat{k}} \right) \\ 
 & C=\frac{1}{2}\frac{u_{b}^{i}u_{b}^{j}}{1-\hat{k}\cdot {{{\hat{u}}}_{b}}}e_{ij}^{A}\left( {\hat{k}} \right) \\ 
\end{align}
$$

### Choose your Polarization Mode!

In [ ]:
eA = zeros(3,3)
ii = 0
for i in range (3):
    for j in range (3):
        eA[ii] = m[i] * m[j] - n[i] * n[j] # Plus Mode
        #eA[ii] = m[i] * n[j] + m[j] * n[i] # Cross Mode
        #eA[ii] = m[i] * m[j] + n[i] * n[j] #Breathing Mode
        #eA[ii] = m[i] * omega[j] + omega[i] * m[j] #Vector Longitudenial Mode 1
        #eA[ii] = n[i] * omega[j] + omega[i] * n[j] #Vector Longitudenial Mode 2
        #eA[ii] = omega[i] * omega[j] #SL
        ii = ii+1
        
B = zeros(3,3)
ii = 0
for i in range (3):
    for j in range (3):
        B[ii] = (u_a[i] * u_a[j]) * sympy.Rational(1,2) * 1/(1 + np.dot(u_a,omega)) * eA[ii]
        ii = ii+1

C = zeros(3,3)
ii = 0
for i in range (3):
    for j in range (3):
        C[ii] = (u_b[i] * u_b[j]) * sympy.Rational(1,2) * 1/(1 + np.dot(u_b,omega)) * eA[ii]
        ii = ii+1
        
D = factor(np.sum(B) * np.sum(C))
print('The Integrand is: ')
D

## Integrating the above over a unit sphere gives you your ORF!

In [ ]:
eA

In [ ]:
B

In [ ]:
C

# ***Question 3***:  Try finding the integrand for each of the given polarization modes. Do you see anything unusual? Can you explain why some modes give an ORF of zero?

# ***Question 4***:  Can you do the integral for all of the given polarizations? Which one do you think is the easiest to do?